# AG_NEWS preprocessing

## Import

In [ ]:
!pip install torchdata
!pip install portalocker==2.8.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import torch
from torchtext.data.functional import to_map_style_dataset
from torchtext.datasets import DATASETS

from transformers import BertModel, BertTokenizer

import numpy as np
from tqdm import tqdm
import pickle as pkl
import os

%cd /src
from utils import extract_features, get_ds_items, save_pkl, load_pkl
%cd ..

## Tokenizer and Feature extractor

In [ ]:
# Load the pre-trained BERT model and tokenizer
BERT_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
BERT_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Data

### Loading original data

In [ ]:
ds = "AG_NEWS"

train_iter = DATASETS[ds](root='./data', split="train")
test_iter = DATASETS[ds](root='./data', split="test")

# train_iter, test_iter = DATASETS[ds]()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

In [ ]:
len(list(train_dataset)), len(list(test_dataset))

(120000, 7600)

### Creat validation set

In [ ]:
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [0.8, 0.2])

In [ ]:
len(list(train_dataset)), len(list(valid_dataset)), len(list(test_dataset))

(96000, 24000, 7600)

### Preprocessing

In [ ]:
train_ds = get_ds_items(train_dataset, BERT_model, BERT_tokenizer)
valid_ds = get_ds_items(valid_dataset, BERT_model, BERT_tokenizer)
test_ds = get_ds_items(test_dataset, BERT_model, BERT_tokenizer)

100%|██████████| 96000/96000 [1:34:47<00:00, 16.88it/s]



ds text len: 96000
ds labels len: 96000	, unique labels len: 4	, labels: {0, 1, 2, 3}
ds features len: 96000	, max: 1.0	, min: -1.0



100%|██████████| 24000/24000 [23:33<00:00, 16.98it/s]



ds text len: 24000
ds labels len: 24000	, unique labels len: 4	, labels: {0, 1, 2, 3}
ds features len: 24000	, max: 1.0	, min: -1.0



100%|██████████| 7600/7600 [07:28<00:00, 16.96it/s]



ds text len: 7600
ds labels len: 7600	, unique labels len: 4	, labels: {0, 1, 2, 3}
ds features len: 7600	, max: 1.0	, min: -1.0



### Saving (and loading)

In [ ]:
ds_dir = "/data/AG_NEWS"

In [ ]:
split = "train"
save_pkl(train_ds, ds_dir, split)

split = "valid"
save_pkl(valid_ds, ds_dir, split)

split = "test"
save_pkl(test_ds, ds_dir, split)

In [ ]:
test_ds[2][0].size()

torch.Size([3072])

In [ ]:
# split = "train"
# t_ds = load_pkl(ds_dir, split)

# split = "valid"
# v_ds = load_pkl(ds_dir, split)

# split = "test"
# te_ds = load_pkl(ds_dir, split)